Pour charger et préparer vos fichiers `badqueries.txt` et `goodqueries.txt` pour l'entraînement d'un modèle de machine learning avec Python et scikit-learn, vous pouvez suivre les étapes ci-dessous. Nous allons lire les fichiers, les étiqueter, les combiner en un seul dataframe, puis les préparer pour le processus d'apprentissage.

### Étape 1 : Charger les données

Assurez-vous que Python peut accéder aux fichiers où ils sont stockés sur votre système. Si nécessaire, ajustez le chemin d'accès aux fichiers.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:


# Charger les données
def load_data(filepath, label):
    with open(filepath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = [(line.strip(), label) for line in lines if line.strip()]
    return data

# Emplacements des fichiers
bad_queries_path = 'badqueries.txt'
good_queries_path = 'goodqueries.txt'

# Charger et étiqueter les données
bad_data = load_data(bad_queries_path, 1)  # Étiquette 1 pour les mauvaises requêtes
good_data = load_data(good_queries_path, 0)  # Étiquette 0 pour les bonnes requêtes

# Combinez les données dans un DataFrame
all_data = pd.DataFrame(bad_data + good_data, columns=['query', 'label'])


### Petite exploration des données

In [3]:
print(all_data.columns)
print(all_data.shape)
print(all_data.head())

Index(['query', 'label'], dtype='object')
(1342596, 2)
                                               query  label
0                 /top.php?stuff='uname >q36497765 #      1
1  /h21y8w52.nsf?<script>cross_site_scripting.nas...      1
2  /ca000001.pl?action=showcart&hop=\"><script>al...      1
3  /scripts/edit_image.php?dn=1&userfile=/etc/pas...      1
4                                /javascript/mta.exe      1




### Étape 2 : Préparation des données

Nous diviserons les données en ensembles d'entraînement et de test, puis appliquerons la vectorisation TF-IDF pour convertir les requêtes textuelles en vecteurs numériques.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(all_data['query'], all_data['label'], test_size=0.2, random_state=42)



In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

def custom_tokenizer(text):
    # Utilisez une expression régulière pour considérer les mots et certains symboles spéciaux
    token_pattern = re.compile(r"(?u)\b\w\w+\b|\/|<|>|script|alert|http|\.exe|\.jpg|=")
    return token_pattern.findall(text)

vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, lowercase=False, strip_accents=None)

# Exemple d'utilisation
example_data = [
    "/examples/jsp/cal/search.php?allwords=<br><script>foo</script>&cid=0&title=1&desc=1",
    "/moodle/filter/tex/texed.php?formdata=foo&pathname=foo\"+||+echo+db+4d+5a+50+00+02+00+00+00+04+00+0f+00+ff+ff+00+00+b8+00+00+00+00+00+00+00+40++>>esbq"
]

X = vectorizer.fit_transform(example_data)
print(vectorizer.get_feature_names_out())  # Affiche les termes du vocabulaire


['/' '00' '02' '04' '0f' '40' '4d' '50' '5a' '<' '=' '>' 'allwords' 'b8'
 'br' 'cal' 'cid' 'db' 'desc' 'echo' 'esbq' 'examples' 'ff' 'filter' 'foo'
 'formdata' 'jsp' 'moodle' 'pathname' 'php' 'script' 'search' 'tex'
 'texed' 'title']


In [7]:
# Vectorisation TF-IDF

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

Pour comprendre à quoi ressemble `X_train_tfidf` après vectorisation avec TF-IDF, vous pouvez visualiser quelques aspects de cette matrice. `X_train_tfidf` est une matrice sparse (creuse) de `scikit-learn`, ce qui signifie qu'elle stocke uniquement les emplacements et les valeurs des éléments non nuls pour économiser de l'espace mémoire.

### 1. Propriétés de base de la matrice


In [8]:
print("Shape of X_train_tfidf:", X_train_tfidf.shape)  # dimensions de la matrice
print("Type of X_train_tfidf:", type(X_train_tfidf))   # type de l'objet (sparse matrix)

Shape of X_train_tfidf: (1074076, 755132)
Type of X_train_tfidf: <class 'scipy.sparse._csr.csr_matrix'>


### 2. Vérifier le nombre d'éléments non nuls

In [9]:
print("Number of non-zero elements:", X_train_tfidf.nnz)
print("Density of the matrix:", X_train_tfidf.nnz / float(X_train_tfidf.shape[0] * X_train_tfidf.shape[1]))

Number of non-zero elements: 3318992
Density of the matrix: 4.0921196018022575e-06


### 3. Affichage des termes du vocabulaire

In [10]:
feature_names = vectorizer.get_feature_names_out()
print("Some feature names:", feature_names[:20])  # affiche les 20 premiers termes du vocabulaire

Some feature names: ['.exe' '.jpg' '/' '00' '000' '0000' '000000' '0000000' '00000000'
 '000000000' '00000000000000000020195001' '00000000000000097vascript'
 '000000000001' '0000000001' '0000000001_000000000000000051240'
 '0000000001_000000000000000122701' '000000000663' '000000000667'
 '000000000699' '000000000788']


Votre mise en œuvre du `TfidfVectorizer` avec un tokenizer personnalisé semble bien adaptée pour gérer les requêtes potentiellement malveillantes que vous avez mentionnées. Vous avez bien configuré les étapes pour diviser vos données et appliquer la vectorisation TF-IDF, ce qui est crucial pour la suite de l'analyse avec un modèle de machine learning.

### Points à considérer :
1. **Validation du Tokenizer :** Assurez-vous que le tokenizer personnalisé extrait bien les termes pertinents de vos données d'entraînement et de test. Les termes imprimés depuis `example_data` donnent une bonne idée des tokens que votre vectorisateur considère.

2. **Entraînement et transformation :** Vous avez utilisé `fit_transform` sur `X_train` et `transform` sur `X_test`, ce qui est la méthode correcte pour éviter la fuite de données de l'ensemble de test dans le modèle de vectorisation.

3. **Gestion des caractères et motifs spéciaux :** Votre expression régulière semble bien conçue pour capturer à la fois les mots standards et les motifs spécifiques aux scripts malveillants, ce qui est essentiel pour ce type d'analyse.

### Étapes suivantes :

Maintenant que vos données sont prêtes, l'étape suivante consiste à choisir et entraîner un modèle de machine learning pour classifier les requêtes comme bonnes ou mauvaises. Voici quelques options de modèles que vous pourriez envisager :

- **Régression logistique :** Un modèle simple mais efficace pour les tâches de classification binaire.
- **Machines à vecteurs de support (SVM) :** Très efficace pour les espaces de grande dimension comme ceux créés par TF-IDF.
- **Forêts aléatoires :** Bon pour gérer des données avec beaucoup de variance et pour capturer des non-linéarités sans trop de tuning nécessaire.
- **Réseaux de neurones :** Si vous avez suffisamment de données et de ressources de calcul, un réseau de neurones pourrait capter des interactions complexes entre les mots.

Chaque modèle a ses avantages et ses inconvénients, et le choix peut dépendre de la taille de vos données, de la précision requise, du temps de calcul disponible, et de votre familiarité avec les modèles.

Si vous souhaitez des conseils sur l'entraînement d'un modèle spécifique ou sur l'évaluation de la performance du modèle, n'hésitez pas à demander !

### Régression logistique

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Création du modèle de régression logistique
logreg = LogisticRegression(random_state=42)

# Entraînement du modèle avec les données d'entraînement
logreg.fit(X_train_tfidf, y_train)

# Prédiction des étiquettes sur l'ensemble de test
y_pred = logreg.predict(X_test_tfidf)

# Calcul et affichage des métriques de performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9944212721584984
Confusion Matrix:
 [[258575    446]
 [  1052   8447]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    259021
           1       0.95      0.89      0.92      9499

    accuracy                           0.99    268520
   macro avg       0.97      0.94      0.96    268520
weighted avg       0.99      0.99      0.99    268520



Votre modèle de régression logistique semble obtenir d'excellents résultats globalement, comme le montre la très haute précision globale (accuracy) de 99.44%. Cependant, il est toujours utile de regarder un peu plus en détail chaque métrique pour bien comprendre les performances du modèle, surtout dans le contexte de la classification de requêtes potentiellement malveillantes.

### Analyse des Résultats

1. **Précision globale (Accuracy) :** À 99.44%, la plupart des prédictions sont correctes. C'est une bonne indication que le modèle fonctionne bien en général.

2. **Matrice de confusion :** 
   - **Vrais positifs pour la classe 0 (bonnes requêtes)** : 258575
   - **Faux positifs pour la classe 0** : 446
   - **Faux négatifs pour la classe 0** : 1052
   - **Vrais positifs pour la classe 1 (mauvaises requêtes)** : 8447

   Les faibles nombres de faux positifs et de faux négatifs suggèrent que le modèle est à la fois précis et sensible. Cependant, le nombre de faux négatifs est supérieur aux faux positifs, ce qui indique une légère faiblesse dans la capture de toutes les mauvaises requêtes (moins de sensibilité).

3. **Rapport de classification :**
   - **Précision pour la classe 0** : Pratiquement parfaite à 1.00.
   - **Précision pour la classe 1** : À 0.95, c'est très bon, mais cela indique aussi que certains faux positifs sont présents.
   - **Rappel pour la classe 0** : Aussi pratiquement parfait à 1.00.
   - **Rappel pour la classe 1** : À 0.89, ce qui indique que le modèle manque environ 11% des mauvaises requêtes réelles.
   - **F1-score pour la classe 1** : À 0.92, ce score équilibré entre précision et rappel est assez bon, mais peut être amélioré, surtout si l'identification précise des mauvaises requêtes est critique.

### Suggestions pour l'amélioration

- **Ajustement des hyperparamètres :** Vous pouvez tenter d'ajuster le paramètre de régularisation `C` de la régression logistique pour voir si une valeur plus faible (plus de régularisation) ou plus élevée (moins de régularisation) améliore les performances.

- **Équilibrage des classes :** Si votre jeu de données est très déséquilibré, envisagez des techniques comme le rééchantillonnage (oversampling de la classe minoritaire ou undersampling de la classe majoritaire) ou l'utilisation de poids de classe dans le modèle de régression logistique pour donner plus de poids à la classe minoritaire pendant l'entraînement.

- **Validation croisée :** Utiliser une validation croisée pour évaluer la robustesse du modèle sur différents sous-ensembles de vos données peut aider à garantir que les performances ne sont pas spécifiques à une division particulière des données.

- **Essayer d'autres modèles :** Parfois, un modèle différent peut capturer les dynamiques des données plus efficacement. Les machines à vecteurs de support (SVM), les forêts aléatoires ou même un modèle de réseaux de neurones pourraient être testés pour comparer les performances.


## SVM
Les machines à vecteurs de support (SVM, pour Support Vector Machines) sont particulièrement efficaces pour les problèmes de classification à haute dimensionnalité, comme c'est souvent le cas avec les données textuelles transformées via TF-IDF. Les SVM sont bien connus pour leur capacité à créer une frontière de décision optimale (appelée hyperplan) qui maximise la marge entre les classes de données.

Pour appliquer un SVM à vos données, vous pouvez utiliser la bibliothèque `scikit-learn`, qui offre une implémentation efficace à travers la classe `SVC` (C-Support Vector Classification). Voici comment vous pouvez mettre en œuvre et évaluer un SVM sur vos données:

### 1. Importer les bibliothèques nécessaires
Vous aurez besoin de `SVC` pour le modèle SVM et de quelques métriques pour évaluer les performances:


In [11]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


### 2. Création et entraînement du modèle SVM
La création d'un modèle SVM avec des paramètres par défaut et son entraînement peut prendre du temps, surtout si la taille de l'ensemble des données est grande. Vous pouvez commencer avec les paramètres par défaut, puis ajuster selon les besoins:

### 3. Prédiction et évaluation du modèle
Après l'entraînement, utilisez le modèle pour faire des prédictions sur l'ensemble de test, puis évaluez les performances:

In [ ]:
# Prédiction des étiquettes sur l'ensemble de test
# y_pred_svm = svm_model.predict(X_test_tfidf)

# # Calcul et affichage des métriques de performance
# print("Accuracy:", accuracy_score(y_test, y_pred_svm))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
# print("Classification Report:\n", classification_report(y_test, y_pred_svm))

### Ajustements possibles
- **Kernel :** Le choix du noyau (`kernel`) est crucial. Les noyaux communs incluent `linear`, `poly`, `rbf`, `sigmoid`. Pour les données textuelles, `linear` est souvent un bon point de départ.
- **C (paramètre de régularisation) :** Ajuster ce paramètre peut aider à contrôler le compromis entre l'atteinte d'une marge maximale et la minimisation de l'erreur de classification.
- **gamma :** Ce paramètre définit l'influence d'un seul exemple d'entraînement, important surtout pour les noyaux non linéaires comme `rbf`.

### Points à noter
- **Scalabilité :** Les SVM peuvent ne pas être les plus rapides pour de très grands ensembles de données à cause de leur complexité de calcul. Dans ce cas, des stratégies comme réduire la taille de l'ensemble de données ou utiliser des versions plus simplifiées des SVM (comme LinearSVC) peuvent être envisagées.
- **Interprétabilité :** Bien que très performants, les SVM ne fournissent pas une interprétabilité aussi directe que d'autres modèles (comme la régression logistique).

N'hésitez pas à expérimenter avec ces paramètres pour optimiser les performances de votre modèle SVM sur votre problème spécifique. Si vous avez besoin d'aide pour ajuster ces paramètres ou pour comprendre leurs effets, je suis là pour aider !


### Mise en place de K-nearest neighbors
KNN est un algorithme d'apprentissage supervisé simple et souvent efficace, qui prédit l'étiquette d'une donnée en se basant sur les étiquettes des "K" échantillons les plus proches dans l'espace des caractéristiques.

### Considérations pour l'utilisation de KNN avec des données textuelles

- **Dimensionnalité et performance** : KNN peut être moins efficace en termes de temps de calcul et de mémoire sur des ensembles de données de haute dimension, comme c'est souvent le cas avec les données textuelles transformées par TF-IDF. La réduction de dimension avec des techniques comme PCA (Principal Component Analysis) pourrait aider à améliorer les performances.
- **Choix de K** : Le nombre de voisins, K, est un paramètre crucial. Un K trop petit peut rendre le modèle sensible au bruit, tandis qu'un K trop grand peut le rendre insensible aux structures réellement présentes dans les données. Il peut être utile de tester plusieurs valeurs de K pour trouver un bon équilibre.
- **Métrique de distance** : La métrique utilisée pour calculer la distance entre les points est également importante. La distance euclidienne est standard, mais pour les données textuelles, d'autres métriques comme la distance de cosine pourraient parfois être plus appropriées.

Expérimentez avec ces paramètres pour voir comment ils influencent les performances du modèle KNN sur vos données. Si vous avez besoin d'aide avec ces ajustements ou d'autres questions sur KNN, n'hésitez pas à demander !

Oui, effectuer une réduction de dimension avant d'utiliser l'algorithme K-nearest neighbors (KNN) est une très bonne idée, surtout pour des données de haute dimension comme celles transformées par TF-IDF. La réduction de dimension peut améliorer significativement la performance de calcul de KNN et potentiellement aussi la qualité de la classification, en réduisant le bruit et en mettant en évidence les structures sous-jacentes des données.

### Utilisation de PCA (Principal Component Analysis)

Une méthode populaire de réduction de dimension est l'analyse en composantes principales (PCA), qui est souvent utilisée pour diminuer le nombre de dimensions tout en conservant le plus d'information possible. Voici comment vous pouvez appliquer PCA à vos données textuelles vectorisées avant de les utiliser pour KNN :

1. **Importation des bibliothèques nécessaires** :
   Vous aurez besoin de `PCA` de la bibliothèque `scikit-learn` ainsi que des autres composants utilisés précédemment.


In [12]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

2. **Application de PCA** :
   Vous devrez choisir le nombre de composantes principales à conserver. Ce choix peut dépendre de la variabilité des données que vous souhaitez conserver.


In [14]:
from sklearn.decomposition import TruncatedSVD

# Création d'une instance de TruncatedSVD pour réduire à 100 dimensions
svd = TruncatedSVD(n_components=100)

# Adapter TruncatedSVD aux données d'entraînement et les transformer
X_train_svd = svd.fit_transform(X_train_tfidf)

# Transformer également les données de test en utilisant le même transformateur
X_test_svd = svd.transform(X_test_tfidf)


3. **Entraînement et évaluation du modèle KNN** :
   Entraînez ensuite votre modèle KNN sur les données réduites.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Exemple de préparation des données
X_train_tfidf_dense = np.array(X_train_tfidf.todense())  # Conversion de sparse à dense

# Supposons que vous avez déjà divisé vos données:
# X_train, X_test, y_train, y_test

# Puis, par exemple pour un modèle RNN :
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model_rnn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_tfidf_dense.shape[1],)),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_rnn.fit(X_train_tfidf_dense, y_train, epochs=10, batch_size=32)
